In [1]:
!pip install tensorboardX

In [2]:
!pip3 install pytorch-ignite

In [31]:
%matplotlib inline
import os
import torch
import sys
import re
import cv2 as cv
from skimage import io, transform
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.nn import MSELoss, Sequential, Linear, Sigmoid, Tanh, L1Loss, Module, Parameter, LeakyReLU, Conv2d, BatchNorm2d, MaxPool2d, Dropout, ReLU
from torch.autograd import Variable
from torch.utils.data import ConcatDataset
from skimage.transform import rotate, warp, AffineTransform
# Ignore warnings
import warnings

In [32]:
try:
    from tensorboardX import SummaryWriter
except ImportError:
    raise RuntimeError("No tensorboardX package is found. Please install with the command: \npip install tensorboardX")

from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss

In [33]:
from transformers import ViTFeatureExtractor, ViTModel

In [34]:
class Dataset(Dataset):

    def __init__(self, root_dir, files, transform=None):
        self.root_dir = root_dir
        self.files = files
        self.transform = transform
        

    def __len__(self):
        return len(self.files)

    
    def __getitem__(self, idx):
        
        img_name = os.path.join(self.root_dir, self.files[idx])
        regression_value = self.files[idx].split('_')[1]
            
        val = float(re.sub(r'[^\d.]', "", regression_value))
        target = val
        image = np.load(img_name)
        sample = torch.tensor(image), torch.tensor(target)

        return sample

In [35]:
class ViTDataset(Dataset):

    def __init__(self, root_dir, files, feature_extractor):
        self.root_dir = root_dir
        self.files = files
        self.feature_extractor = feature_extractor
        self.W, self.H = 224, 224
        

    def __len__(self):
        return len(self.files)

    
    def __getitem__(self, idx):
        
        img_name = os.path.join(self.root_dir, self.files[idx])
        regression_value = self.files[idx].split('_')[1]
            
        val = float(re.sub(r'[^\d.]', "", regression_value))
        target = val
        image = np.load(img_name)
        image = image[1:4]
        inpt = self.feature_extractor(images=image, return_tensors="pt")
        channels = 3
        pixel_values = inpt['pixel_values'].view(channels, self.feature_extractor.size, self.feature_extractor.size)
        
        sample = pixel_values, torch.tensor(target)

        return sample

In [140]:
size = 0.9

In [141]:
root_dir = "../input/oilalldata/no_oil"

In [142]:
files = os.listdir(root_dir)
train_files = files[:int(size*len(files))]
test_files = files[int(size*len(files)):]

In [143]:
train_dataset = Dataset(root_dir=root_dir,
                        files=train_files)

In [144]:
test_dataset = Dataset(root_dir=root_dir,
                        files=test_files)

In [145]:
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

In [146]:
train_dataset_vit = ViTDataset(root_dir=root_dir,
                        files=train_files, feature_extractor=feature_extractor)

In [147]:
test_dataset_vit = ViTDataset(root_dir=root_dir,
                        files=train_files, feature_extractor=feature_extractor)

In [148]:
train_loader_vit = DataLoader(
    train_dataset_vit, batch_size=8, shuffle=True
)

test_loader_vit = DataLoader(
    test_dataset_vit, batch_size=8, shuffle=False
)

In [149]:
def get_data_loaders(train_batch_size=8, val_batch_size=64):

    train_loader = DataLoader(
        train_dataset, batch_size=train_batch_size, shuffle=True
    )

    test_loader = DataLoader(
        test_dataset, batch_size=val_batch_size, shuffle=False
    )

    return train_loader, test_loader

In [150]:
class ViT(Module):
    def __init__(self, n_classes: int, ViT_model, n_hidden_layers_cat=12):
        super().__init__()
        self.n_hidden_layers_cat = n_hidden_layers_cat
        self.ViT_model = ViT_model
        self.dropout = Dropout(0.1)
        self.classifier = Linear(self.ViT_model.config.hidden_size, n_classes)
    
    
    def forward(self, pixel_values):
        output = self.ViT_model(pixel_values)
        last_hidden_state = output.last_hidden_state
        pooler_output = output.pooler_output # but we concat last hidden_layers
        #print('Pooler output: ', pooler_output.shape)
        hidden_states = output.hidden_states
        #print('Hidden_states ', len(hidden_states))
        summed_last_cat_layers = torch.stack(hidden_states[-self.n_hidden_layers_cat:]).sum(0)
        pooled_vector = torch.mean(summed_last_cat_layers, dim=1) # may be better than sum(), or we can use max-pooling
        pooled_vector = self.dropout(pooled_vector)
        #print('pooled_vector: ', pooled_vector.shape)
        output = self.classifier(pooled_vector)
        # сразу сделаем reshape 
        output = output.view(-1, 1)
        #print('out: ', output.shape)
        return output

In [151]:
class Net_res(Module):
    def __init__(self):
        super().__init__()
        # input 14 x 64 x 64
        self.layer1 = Sequential(
            # input 14 x 64 x 64
            Conv2d(14, 28, 3, 1, 1, bias=False),
            LeakyReLU(0.1, inplace=True),
            BatchNorm2d(28),
            # state size 28 x 64 x 64
            Conv2d(28, 28, 3, 1, 1, bias=False),
            LeakyReLU(0.1, inplace=True),
            BatchNorm2d(28),
            MaxPool2d(2, 2))
        self.layer2 = Sequential(
            Conv2d(28, 56, 3, 1, 1, bias=False),
            LeakyReLU(0.1, inplace=True),
            BatchNorm2d(56),
            # state size 56 x 32 x 32
            Conv2d(56, 56, 3, 1, 1, bias=False),
            LeakyReLU(0.1, inplace=True),
            BatchNorm2d(56),
            MaxPool2d(2, 2))
        self.layer3 = Sequential(
            Conv2d(56, 112, 3, 1, 1, bias=False),
            LeakyReLU(0.1, inplace=True),
            BatchNorm2d(112),
            # state size 112 x 16 x 16
            Conv2d(112, 112, 3, 1, 1, bias=False),
            LeakyReLU(0.1, inplace=True),
            BatchNorm2d(112),
            MaxPool2d(2, 2))
        self.layer4 = Sequential(
            Conv2d(112, 128, 3, 1, 1, bias=False),
            LeakyReLU(0.1, inplace=True),
            BatchNorm2d(128),
            # state size 128 x 8 x 8
            Conv2d(128, 128, 3, 1, 1, bias=False),
            LeakyReLU(0.1, inplace=True),
            BatchNorm2d(128),
            MaxPool2d(2, 2))
        self.layer5 = Sequential(
            Conv2d(128, 256, 3, 1, 1, bias=False),
            LeakyReLU(0.1, inplace=True),
            BatchNorm2d(256),
            # state size 256 x 4 x 4
            Conv2d(256, 256, 3, 1, 1, bias=False),
            LeakyReLU(0.1, inplace=True),
            BatchNorm2d(256),
            MaxPool2d(2, 2))
        self.layer6 = Sequential(
            Conv2d(256, 512, 3, 1, 1, bias=False),
            LeakyReLU(0.1, inplace=True),
            BatchNorm2d(512),
            # state size 512 x 2 x 2
            Conv2d(512, 512, 3, 1, 1, bias=False),
            LeakyReLU(0.1, inplace=True),
            BatchNorm2d(512),
            MaxPool2d(2, 2))
        self.fc = Sequential(Linear(512, 128), ReLU(), Dropout(0.1), Linear(128, 1))
    def forward(self, x):
        out1 = self.layer1(x)
        out2 = self.layer2(out1)
        out3 = self.layer3(out2)
        out4 = self.layer4(out3)
        out5 = self.layer5(out4)
        out6 = self.layer6(out5)
        out = self.fc(out6.reshape(x.shape[0], -1))
        return out.view(x.shape[0], 1)

In [152]:
MODEL_NAME = 'google/vit-base-patch16-224-in21k'

In [161]:
ViT_model = ViTModel.from_pretrained(MODEL_NAME, return_dict=True, output_hidden_states=True)

In [170]:
network = Net_res()
train_dataloader, test_dataloader = get_data_loaders()
#network = ViT(n_classes=1, ViT_model=ViT_model)
#train_dataloader, test_dataloader = train_loader_vit, test_loader_vit
writer = SummaryWriter(filename_suffix='first', logdir="logs")

device = "cpu"

if torch.cuda.is_available():
    device = "cuda"
criterion = MSELoss()
optim = torch.optim.Adam
learning_rate = 0.001
max_epochs = 2

network = network.to(device)

optimizer = optim(network.parameters(), lr=learning_rate)

trainer = create_supervised_trainer(network, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(
    network, metrics={"L2 loss": Loss(criterion), "L1 loss": Loss(L1Loss())}, device=device
)

training_history = {'l1':[],'l2':[]}
validation_history = {'l1':[],'l2':[]}
last_epoch = []

In [171]:
device

In [172]:
@trainer.on(Events.ITERATION_COMPLETED(every=10))
def log_training_loss(engine):
    writer.add_scalar("training/loss", engine.state.output, engine.state.iteration)
    writer.flush()


@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    evaluator.run(train_dataloader)
    metrics = evaluator.state.metrics
    l2_train = metrics["L2 loss"]
    l1_train = metrics['L1 loss']
    last_epoch.append(0)
    training_history['l2'].append(l2_train)
    training_history['l1'].append(l1_train)
    writer.add_scalar("training/epoch_loss", l2_train, trainer.state.epoch)
    
    evaluator.run(test_dataloader)
    metrics = evaluator.state.metrics
    l2 = metrics["L2 loss"]
    l1 = metrics['L1 loss']
    validation_history['l2'].append(l2)
    validation_history['l1'].append(l1)
    
    writer.add_scalar("validation/epoch_loss", l2, trainer.state.epoch)
    writer.flush()
    
    sys.stdout.write("\rTraining/Validation Results - Epoch: {}  Avg MSE: {:.5f} / {:.5f} Avg MAE: {:.5f} / {:.5f}"
          .format(trainer.state.epoch, l2_train, l2, l1_train, l1))

In [173]:
trainer.run(train_dataloader, max_epochs=max_epochs)

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(training_history['l2'][1:], label='Train')
plt.plot(validation_history['l2'][1:], label='Test')
plt.xlabel('Epochs', fontsize=16)
plt.ylabel('Loss', fontsize=16)
plt.legend(loc=0, fontsize=16)
plt.grid('on')
plt.show()

In [ ]:
PATH_2_SAVE = "model_my_vit_oil.pt"

In [ ]:
torch.save(network.state_dict(), PATH_2_SAVE)

In [178]:
def predict(img_name, threshold=0.0001):
    image = np.load(img_name)
    image = torch.tensor(image).view(1, image.shape[0], image.shape[1], image.shape[2])
    
    image = image.to(device)
    score = network(image)
    if score.item() > threshold:
        print("Розлив площадью ", score.item(), " га")
    else:
        print("Розлив не найден")

In [179]:
predict("../input/dataset/2017-01out/Classifiacation(1)_regression(0.0247)_ЛУК-2020-4_2.npy")